In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [7]:
from pathlib import Path
import numpy as np

## Model and trial

In [8]:
class NLayerDiscriminator(nn.Module):
    """
    'PatchGAN' classifier described in the original pix2pix paper.
    It can classify whether 70×70 overlapping patches are real or fake.
    Such a patch-level discriminator architecture has fewer parameters
    than a full-image discriminator and can work on arbitrarily-sized images
    in a fully convolutional fashion.
    """
    def __init__(self, input_nc, ndf=64, n_layers=3, norm_layer=nn.InstanceNorm2d):
        """
        Construct a PatchGAN discriminator
        
        Parameters:
            1. input_nc (int): the number of channels in input images
            2. ndf (int): the number filters
            3. n_layers (int): the number of conv layers in the discriminator
            4. norm_layer (nn.Module): normalization layer 
        """
        super(NLayerDiscriminator, self).__init__()
        use_bias = (norm_layer == nn.InstanceNorm2d)
    
        kernel_size, padding = 4, 1
        
        # Leading Conv
        sequence = [
            nn.Conv2d(input_nc, ndf, kernel_size=kernel_size, stride=2, padding=padding),
            nn.LeakyReLU(.2, True)
        ]
        
        # gradually increase the number of filters and decrease the dimension of the image
        mult_i = 1
        for n in range(n_layers):
            mult_o = min(mult_i * 2, 8)
            ic, oc = ndf * mult_i, ndf * mult_o
            sequence += [
                nn.Conv2d(
                    ic, oc, 
                    kernel_size=kernel_size, 
                    stride=2, 
                    padding=padding, 
                    bias=use_bias),
                norm_layer(oc),
                nn.LeakyReLU(.2, True)
            ]
            mult_i = mult_o
        
        mult_o = min(mult_i * 2, 8)
        ic, oc = ndf * mult_i, ndf * mult_o
        sequence += [
            nn.Conv2d(ic, oc, kernel_size=kernel_size, stride=1, padding=padding, bias=use_bias),
            norm_layer(oc),
            nn.LeakyReLU(.2, True)
        ]
        
        sequence += [nn.Conv2d(oc, 1, kernel_size=kernel_size, stride=1, padding=padding)]
        
        self.model = nn.Sequential(*sequence)
        
    def forward(self, x):
        return self.model(x)
     
#     def forward(self, x):
#         for l, layer in enumerate(self.model):
#             y = layer(x)
#             print(type(layer))
#             print(f'\n\tencoder layer = {l}\n\tinput shape = {x.shape}\n\toutput shape = {y.shape}\n')
#             x = y
#         return x

In [95]:
# class NLayerDiscriminator(nn.Module):
#     """
#     'PatchGAN' classifier described in the original pix2pix paper.
#     It can classify whether 70×70 overlapping patches are real or fake.
#     Such a patch-level discriminator architecture has fewer parameters
#     than a full-image discriminator and can work on arbitrarily-sized images
#     in a fully convolutional fashion.
#     """
#     def __init__(self, input_nc, ndf=64, n_layers=3, norm_layer=nn.InstanceNorm2d):
#         """
#         Construct a PatchGAN discriminator
        
#         Parameters:
#             1. input_nc (int): the number of channels in input images
#             2. ndf (int): the number filters
#             3. n_layers (int): the number of conv layers in the discriminator
#             4. norm_layer (nn.Module): normalization layer 
#         """
#         super(NLayerDiscriminator, self).__init__()
#         use_bias = (norm_layer == nn.InstanceNorm2d)
    
#         kernel_size, padding = 3, 0
        
#         # Leading Conv
#         sequence = [
#             nn.Conv2d(input_nc, ndf, kernel_size=kernel_size, stride=2, padding=padding),
#             nn.LeakyReLU(.2, True)
#         ]
        
#         # gradually increase the number of filters and decrease the dimension of the image
#         mult_i = 1
#         for n in range(n_layers):
#             mult_o = min(mult_i * 2, 8)
#             ic, oc = ndf * mult_i, ndf * mult_o
#             sequence += [
#                 nn.Conv2d(
#                     ic, oc, 
#                     kernel_size=kernel_size, 
#                     stride=2, 
#                     padding=padding, 
#                     bias=use_bias),
#                 norm_layer(oc),
#                 nn.LeakyReLU(.2, True)
#             ]
#             mult_i = mult_o
            
#         self.fc1 = nn.Linear(32 * 5 * 5, 80)
#         self.fc2 = nn.Linear(80, 8)
#         self.fc3 = nn.Linear(8, 1)
        
#         self.model = nn.Sequential(*sequence)
        
     
#     def forward(self, x):
#         for l, layer in enumerate(self.model):
#             y = layer(x)
#             # print(type(layer))
#             # print(f'\n\tencoder layer = {l}\n\tinput shape = {x.shape}\n\toutput shape = {y.shape}\n')
#             x = y
        
#         # print(x.shape)
#         x = torch.flatten(x, 1) # flatten all dimensions except batch (start_dim = 1)
#         # print(x.shape)
#         x = F.relu(self.fc1(x))
#         # print(x.shape)
#         x = F.relu(self.fc2(x))
#         # print(x.shape)
#         x = F.relu(self.fc3(x))
#         # print(x.shape)
#         return x

In [9]:
def trial(model, input_shape, cuda=True):
    x = torch.rand(16, 1, *input_shape, dtype=torch.float32)
    if cuda:
        model = model.cuda()
        x = x.cuda()

    y = model.forward(x)
    print(f'Output shape:\n\t{y.shape}\n')

    num_parameters_trainable = sum([p.numel() for p in model.parameters() if p.requires_grad])
    num_parameters = sum([p.numel() for p in model.parameters()])
    print(f'Number of parameters:\n\tTrainable = {num_parameters_trainable}\n\tTotal = {num_parameters}') 

In [10]:
discriminator = NLayerDiscriminator(1, ndf=4, n_layers=4, norm_layer=nn.InstanceNorm2d)
input_shape = [128, 128]
trial(discriminator, input_shape)

Output shape:
	torch.Size([16, 1, 2, 2])

Number of parameters:
	Trainable = 44221
	Total = 44221


## Load data

In [72]:
class Dataset_ls4gan(Dataset):
    """
    LS4GAN dataset
    """
    def __init__(self, class_paths):
        super(Dataset_ls4gan, self).__init__()    
        self.image_fnames = []
        self.labels = []
        for c, class_path in enumerate(class_paths):
            fnames = list(Path(class_path).glob('*npz'))
            self.image_fnames += fnames
            self.labels += [c] * len(fnames)
            
        # print(len(self.image_fnames) == len(self.labels))    
        
        shuffle_indices = np.arange(len(self.image_fnames))
        np.random.shuffle(shuffle_indices)
        self.image_fnames = np.array(self.image_fnames)[shuffle_indices]
        self.labels = np.array(self.labels)[shuffle_indices]
        self.labels = self.labels.astype(np.float32)
        
        
    def __len__(self):
        return len(self.image_fnames)
    
    def __getitem__(self, idx):
        image_fname, label = self.image_fnames[idx], self.labels[idx]
        
        image = np.load(image_fname)
        key = list(image.keys())[0]
        image = image[key]
        image = np.expand_dims(np.float32(image), 0)
        
        image_tensor = torch.from_numpy(image)
        label_tensor = torch.tensor([label])
        return image_tensor, label_tensor

In [89]:
class_paths_train = [
    '/sdcc/u/yhuang2/PROJs/GAN/datasets/ls4gan/toyzero/U/trainA/',
    '/sdcc/u/yhuang2/PROJs/GAN/datasets/ls4gan/toyzero/U/trainB/'
]

class_paths_test = [
    '/sdcc/u/yhuang2/PROJs/GAN/datasets/ls4gan/toyzero/U/testA/',
    '/sdcc/u/yhuang2/PROJs/GAN/datasets/ls4gan/toyzero/U/testB/'
]

bsz = 16
dataset_train = Dataset_ls4gan(class_paths_train)
dataset_test = Dataset_ls4gan(class_paths_test)
train_loader = DataLoader(dataset_train, batch_size=bsz, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=bsz, shuffle=True)

## Train

In [101]:
discriminator = NLayerDiscriminator(1, ndf=4, n_layers=4, norm_layer=nn.InstanceNorm2d)
discriminator = discriminator.cuda()

criterion = nn.BCEWithLogitsLoss()
# criterion = nn.L1Loss()
optimizer = torch.optim.AdamW(discriminator.parameters(), lr=0.0001)
epochs = 200

for epoch in range(epochs):  # loop over the dataset multiple times

    epoch_loss = 0
    for i, data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = discriminator(inputs)
        outputs = outputs.reshape_as(labels)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        epoch_loss += loss.item()
        if i == bsz - 1: 
            print(f'[{epoch + 1}] loss: {epoch_loss / bsz :.3f}')

print('Finished Training')

[1] loss: 0.709
[2] loss: 0.698
[3] loss: 0.694
[4] loss: 0.693
[5] loss: 0.694
[6] loss: 0.694
[7] loss: 0.693
[8] loss: 0.693
[9] loss: 0.693
[10] loss: 0.693
[11] loss: 0.693
[12] loss: 0.693
[13] loss: 0.693
[14] loss: 0.693
[15] loss: 0.693
[16] loss: 0.693
[17] loss: 0.693
[18] loss: 0.693
[19] loss: 0.693
[20] loss: 0.693
[21] loss: 0.693
[22] loss: 0.693
[23] loss: 0.693
[24] loss: 0.694
[25] loss: 0.694
[26] loss: 0.693
[27] loss: 0.693
[28] loss: 0.693
[29] loss: 0.693
[30] loss: 0.693
[31] loss: 0.694
[32] loss: 0.694
[33] loss: 0.693
[34] loss: 0.693
[35] loss: 0.693
[36] loss: 0.693
[37] loss: 0.693
[38] loss: 0.693
[39] loss: 0.693
[40] loss: 0.693
[41] loss: 0.693
[42] loss: 0.693
[43] loss: 0.693
[44] loss: 0.693
[45] loss: 0.693
[46] loss: 0.693
[47] loss: 0.693
[48] loss: 0.693
[49] loss: 0.693
[50] loss: 0.693
[51] loss: 0.693
[52] loss: 0.693
[53] loss: 0.693
[54] loss: 0.693
[55] loss: 0.693
[56] loss: 0.693
[57] loss: 0.693
[58] loss: 0.693
[59] loss: 0.693
[60] l